# Отток клиентов банка

**Введение:**  
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

**Цели и задачи:**  
Построение модели с предельно большим значением *F1*-меры. Необходимо довести метрику до 0.59.

**Входные данные:**  
Исторические сведения об активности клиентов банка. 
Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)


## Обзор исходных данных

Первым делом импортируем необходимые библиотеки и методы.

In [1]:
import pandas as pd
from scipy import stats as st
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.utils import shuffle
import warnings

warnings.filterwarnings('ignore')

Загрузим данные и посмотрим на полученную таблицу. 

In [2]:
df = pd.read_csv('/datasets/Churn.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


База содержит 14 столбцов и 10 тысяч строк. Типы данных соответствуют. Столбец *'Tenure'* содержит пропуски, около 9%. Столбец *'RowNumber'* фактически повторяет индекс, его можно удалить. Фамилии клиентов и их ID для текущей задачи также не нужны, будут только мешаться. 
Типы оставшихся переменных:  
**Категориальные**: 'Geography', 'Gender'  
**Количественные**: 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary'  
**Логические**: 'HasCrCard', 'IsActiveMember', 'Exited'  

In [4]:
print(df['Geography'].unique())

['France' 'Spain' 'Germany']


В базе данных представлены клиенты трёх стран. 

In [5]:
print(df['Gender'].unique())

['Female' 'Male']


Гендерное разнообразие вполне естественное. 

In [6]:
numerical = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
df[numerical].describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary
count,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.997690,76485.889288,1.530200,100090.239881
std,96.653299,10.487806,2.894723,62397.405202,0.581654,57510.492818
min,350.000000,18.000000,0.000000,0.000000,1.000000,11.580000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,51002.110000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,100193.915000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,149388.247500
max,850.000000,92.000000,10.000000,250898.090000,4.000000,199992.480000


Численные переменные в порядке, аномалий пока не выявлено. Однако зафиксирован различный масштаб признаков.

In [7]:
print(df['HasCrCard'].unique())
print(df['IsActiveMember'].unique())
print(df['Exited'].unique())

[1 0]
[1 0]
[1 0]


Логические переменные похожи на таковые. 

**Выводы:**  
Обзор исходных данных показал наличие ~9% пропущенных данных в столбце *Tenure* и лишней колонки *RowNumber*.  
Категориальные признаки требуют кодирования, а количественные - масштабирования.

## Подготовка данных

Сразу удалим лишние столбцы.

In [8]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

### Обработка пропусков

Пропуски имеются только в столбце Tenure и их примерно 9%, это достаточно много. Причём сам признак - сколько лет человек является клиентом банка - может быть весьма значимым для решаемой задачи. Для выбора способа обработки пропусков выясним насколько они случайны. Разделим основной датафрейм две таблицы по наличию пропусков и посмотрим отличаются ли средние значения численных переменных в этих таблицах. 

In [9]:
df['GoodData'] = 0
df['GoodData'] = df['GoodData'].where(df['Tenure'].isna(), 1)

numerical.remove('Tenure')

df.pivot_table(index='GoodData', values=numerical, aggfunc='mean')

,Age,Balance,CreditScore,EstimatedSalary,NumOfProducts
GoodData,,,,,
0,38.647965,76117.341474,648.451045,99180.389373,1.530253
1,38.949181,76522.740015,650.736553,100181.214924,1.530195


Средние значения количественных переменных для "хороших" данных и данных с пропусками близки. Но на всякий случай проведём статистический анализ на основе t-критерия Стьюдента для двух выборок. Нулевую гипотезу для каждого признака сформулируем как Н0: "Средние значения переменной для выборки с пропусками и без пропусков не различаются". Соответствующая нулевой альтернативная гипотеза Н1: "Средние значения переменной для выборки с пропусками и без пропусков различны". Пороговое значение установим стандартное 0.05. Для наглядного представления сформируем таблицу p-значений для каждого признака. 

In [10]:
def p_vals(col):
    samp_1 = df.query('GoodData == 1')[col]
    samp_2 = df.query('GoodData == 0')[col]
    result = st.ttest_ind(samp_1, samp_2).pvalue
    return result

data = [p_vals(x) for x in numerical]
pd.DataFrame(data, numerical).rename(columns={0:'p-value'})

,p-value
CreditScore,0.496686
Age,0.409046
Balance,0.851853
NumOfProducts,0.997700
EstimatedSalary,0.616914


Во всех случаях p-значение значительно выше порогового уровня, то есть отклонить нулевую гипотезу оснований нет. 

Доля "хороших" данных в категориальных признаках:

In [11]:
df.pivot_table(index='Gender', values='GoodData', aggfunc='mean')

,GoodData
Gender,
Female,0.906229
Male,0.911490


In [12]:
df.pivot_table(index='Geography', values='GoodData', aggfunc='mean')

,GoodData
Geography,
France,0.907459
Germany,0.913910
Spain,0.907549


Существенной разницы не наблюдается. С учётом результатов статистического анализа для численных переменных можно сделать вывод, что пропуске в столбце *'Tenure'* являются случайными и их удаление не исказит общую картину, поэтому строки с пропущенными данными мы удалим. 

In [13]:
df = df.dropna().drop('GoodData', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 11 columns):
CreditScore        9091 non-null int64
Geography          9091 non-null object
Gender             9091 non-null object
Age                9091 non-null int64
Tenure             9091 non-null float64
Balance            9091 non-null float64
NumOfProducts      9091 non-null int64
HasCrCard          9091 non-null int64
IsActiveMember     9091 non-null int64
EstimatedSalary    9091 non-null float64
Exited             9091 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 852.3+ KB


От пропусков избавились. 

### Кодирование признаков

Для правильной работы алгоритмов машинного обучения необходимо преобразовать строковые переменные в числовые. Для этого воспользуемся техникой прямого кодирования.

In [14]:
df = pd.get_dummies(df, drop_first=True)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Сделано.

### Разделение на выборки

Разделим массив данных на обучающую, валидационную и тестовую выборки в соотношении 60% - на обучение модели, 20% - на валидацию и оставшиеся 20% на итоговое тестирование. 

In [15]:
train, test = train_test_split(df, test_size = 0.4, random_state=12345)
test, valid = train_test_split(test, test_size = 0.5, random_state=12345)

In [16]:
len(train) + len(test) + len(valid)

9091

Контрольная сумма совпала.  
Разделим признаки и целевую переменную.

In [17]:
target = 'Exited'
features = df.drop(target, axis=1).columns

In [18]:
features_train = train[features].copy()
target_train = train[target].copy()
features_valid = valid[features].copy()
target_valid = valid[target].copy()
features_test = test[features].copy()
target_test = test[target].copy()

Таким образом данные разделены на три выборки: обучающую, валидациционную и тестовую в соотношении 3:1:1. 

### Масштабирование признаков 

Преобразуем масштаб численных признаков методом стандартизации. Для чистоты эксперимента настройку стандартизатора будем проводить на обучающей выборке, поскольку в реальной задаче тестовая выборка может быть недоступна. 

In [19]:
scaler = StandardScaler()
scaler.fit(features_train[numerical])

features_train[numerical] = scaler.transform(features_train[numerical])
features_valid[numerical] = scaler.transform(features_valid[numerical])
features_test[numerical] = scaler.transform(features_test[numerical])

In [19]:
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9344,727,28,2.0,110997.76,1,1,0,101433.76,0,0,0
3796,537,26,7.0,106397.75,1,0,0,103563.23,0,0,1
7462,610,40,9.0,0.00,1,1,1,149602.54,0,0,1
1508,576,36,6.0,0.00,2,1,1,48314.00,0,0,1
4478,549,31,4.0,0.00,2,0,1,25684.85,0,0,1


**Выводы:**  
Проведена подготовка данных:  
- Удалены ненужные столбцы. 
- Обработаны пропуски путём удаления. 
- Выполнена кодировка категориальных признаков. 
- Численные признаки отмасштабированы. 
- Данные разделены на выборки: обучающую, валидационную и тестовую. 

## Баланс классов. Пробное обучение

Исследуем баланс классов на тренировочной выборке. 

In [20]:
target_train.mean()

0.20645397873120647

Доля класса "1", то есть ушедших клиентов, составляет примерно 20% от всей выборки. Значит имеет место дисбаланс классов, что может негативно сказаться на качестве модели.

Тем не менее, попробуем обучить модель на том, что есть, и посмотрим, какое качество мы получаем. В первом приближении обучим модели логистической регрессии и случайного леса без дополнительных настроек. Оценку качества будем проводить на основании метрик F1 и AUC ROC. 

In [21]:
def scoring(features, target):
    """
    Функция вычисляет метрики F1 и AUC-ROC для актуальной модели 
    и выводит результат
    """
    global model
    predictions = model.predict(features)
    f1 = f1_score(target, predictions)
    prob_valid = model.predict_proba(features)
    prob_one_valid = prob_valid[:, 1]
    auc_roc = roc_auc_score(target, prob_one_valid)
    print('F1 = {:.2f} | AUC-ROC = {:.2f}'.format(f1, auc_roc))

Логистическая регрессия:

In [22]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)

scoring(features_valid, target_valid)

F1 = 0.08 | AUC-ROC = 0.68


Случайный лес:

In [23]:
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)

scoring(features_valid, target_valid)

F1 = 0.54 | AUC-ROC = 0.82


Получили F1-меру для логистической регрессии 0.32, а для случайного леса 0.54, что меньше, чем требуется по условиям задачи, однако модели работают, что следует из величины AUC-ROC, которая значительно выше, чем 0.5, то есть даже без подбора гиперпараметров предсказывают результат лучше, чем случайная классификация. 

**Выводы:**  
- В рассматриваемой задаче классы целевого признака несбалансированы в соотношении 4:1, т.е. искомый класс "ушедший клиент" составляет всего 20% от выборки.  
- Обучение модели случайного леса "в лоб" без настройки гиперпараметров и балансировки классов показывает величину F1 = 0.54, AUC_ROC = 0.82. Логистическая регрессия показывает более скромный результат.  
- Для увеличения качества модели требуется балансировка классов и подбор гиперпараметров. 

## Улучшение модели

План работы следующий:  
1) учесть несбалансированность классов различными методами,  
2) попробовать другие модели и выбрать лучшую,  
3) подобрать гиперпараметры выбранной модели. 

### Балансировка классов

Рассмотрим три метода работы с несбалансированной классификацией: взвешивание классов, увеличение и уменьшение выборки. Выберем наилучший вариант. 

#### Взвешивание классов

В параметрах модели определим аргумент *class_weight*, который отвечает за вес каждого класса. 

In [24]:
model = LogisticRegression(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
print('Логистическая регрессия:')
scoring(features_valid, target_valid)

model = RandomForestClassifier(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
print('\nСлучайный лес:')
scoring(features_valid, target_valid)

Логистическая регрессия:
F1 = 0.49 | AUC-ROC = 0.78

Случайный лес:
F1 = 0.53 | AUC-ROC = 0.82


Качество логистической регрессии со взвешиванием классов заметно увеличилось. А вот "случайный лес" без изменений, F1 даже слегка уменьшился. 

#### Увеличение выборки

В тренировочной выборке искуственно увеличим количество строк положительного класса.  
Рассмотрим результат в зависимости от коэффициента мультипликации положительного класса: 2, 3, 4, 5. Будем рассматривать только случайный лес, потому что логистическая регрессия даёт стабильно худший результат. 

In [25]:
def upsample(features, target, repeat):
    """Функция выделяет объекты класса '1', 
    умножает их количество на заданный коэффициент, 
    перемешивает данные и возвращает увеличенную выборку. 
    """
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

mult_coef = [2, 3, 4, 5]

for i in mult_coef:
    features_upsampled, target_upsampled = upsample(features_train, target_train, i)

    model = RandomForestClassifier(random_state=12345)
    model.fit(features_upsampled, target_upsampled)
    
    if i < 5:
        print(f'Класс "1" увеличен в {i} разa:')
    else:  
        print(f'Класс "1" увеличен в {i} раз:')
    print('Баланс классов: {:.2f}'.format(target_upsampled.mean()))
    scoring(features_valid, target_valid)
    print()

Класс "1" увеличен в 2 разa:
Баланс классов: 0.34
F1 = 0.55 | AUC-ROC = 0.82

Класс "1" увеличен в 3 разa:
Баланс классов: 0.44
F1 = 0.55 | AUC-ROC = 0.82

Класс "1" увеличен в 4 разa:
Баланс классов: 0.51
F1 = 0.59 | AUC-ROC = 0.82

Класс "1" увеличен в 5 раз:
Баланс классов: 0.57
F1 = 0.55 | AUC-ROC = 0.82



Лучший результат F1 = 0.59 получился при увеличении выборки путём повторения объектов положительного класса 4 раза, то есть в случае, когда количество объектов классов "0" и "1" примерно одинковое. 

#### Уменьшение выборки

Теперь посмотрим как на качество модели влияет обратное действие - уменьшение выборки. Уменьшать будем количество объектов отрицательного класса "0", так же - в несколько шагов до достижения паритета между классами. 

In [26]:
def downsample(features, target, fraction):
    """Функция выделяет объекты класса '0', 
    делает из них выборку с заданным коэффициентом, 
    перемешивает данные и возвращает уменьшенную выборку. 
    """
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

frac_coef = [0.8, 0.5, 0.3, 0.25, 0.2]

for i in frac_coef:
    features_downsampled, target_downsampled = downsample(features_train, target_train, i)

    model = RandomForestClassifier(random_state=12345)
    model.fit(features_downsampled, target_downsampled)
    
    print(f'Доля выборки класса "0": {i}')
    print('Баланс классов: {:.2f}'.format(target_downsampled.mean()))
    scoring(features_valid, target_valid)
    print()

Доля выборки класса "0": 0.8
Баланс классов: 0.25
F1 = 0.56 | AUC-ROC = 0.82

Доля выборки класса "0": 0.5
Баланс классов: 0.34
F1 = 0.58 | AUC-ROC = 0.84

Доля выборки класса "0": 0.3
Баланс классов: 0.46
F1 = 0.59 | AUC-ROC = 0.83

Доля выборки класса "0": 0.25
Баланс классов: 0.51
F1 = 0.54 | AUC-ROC = 0.83

Доля выборки класса "0": 0.2
Баланс классов: 0.57
F1 = 0.53 | AUC-ROC = 0.82



В данном случае лучший результат получается, если из обучающей выборки выбросить половину объектов класса "0", когда баланс классов равен 0.34. При попытке удалить больше качество модели снижается вследствие недообучения.  

**Промежуточный итог.** Увеличение выборки показывает лучший результат чем уменьшение. Взвешивание классов оказывает положительный эффект только на модель логистической регрессии. 

### Выбор модели и гиперпараметров

На предыдущем шаге получено, что увеличение выборки показывает наилучшее качество на валидационной выборке. Зафиксируем это, сформировав новую обучающую выборку.  

In [27]:
features_train, target_train = upsample(features_train, target_train, 4)

Теперь обучим на ней несколько моделей и выберем модель с наилучшим качеством.  

Начнём с модели логистической регрессии. В ней варьировать особо нечего. 

In [28]:
solvers = ['lbfgs', 'liblinear', 'sag', 'saga']

for solver in solvers:
    model = LogisticRegression(random_state=12345, solver=solver, max_iter=10000)
    model.fit(features_train, target_train)
    print(f'Солвер: {solver}:')
    scoring(features_valid, target_valid)

Солвер: lbfgs:
F1 = 0.42 | AUC-ROC = 0.71
Солвер: liblinear:
F1 = 0.43 | AUC-ROC = 0.72
Солвер: sag:
F1 = 0.36 | AUC-ROC = 0.63
Солвер: saga:
F1 = 0.35 | AUC-ROC = 0.61


Как уже сказано выше, в данной задаче логистическая регрессия показывает далеко не лучший результат.  

Попробуем модель дерева решений с различными гиперпараметрами. 

In [29]:
best_tree_model = None
best_tree_result = 0

max_depth = 10
samples_split = 6
samples_leaf = 10
best_leaf = 0
best_split = 0

for depth in range(1, max_depth+1):
    for split in range(2, samples_split+1):
        for leaf in range(1, samples_leaf+1):
            model = DecisionTreeClassifier(random_state=12345, 
                                           max_depth=depth, 
                                           min_samples_split=split, 
                                           min_samples_leaf=leaf)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            result = f1_score(target_valid, predictions)
            if result > best_tree_result:
                best_tree_model = model
                best_tree_result = result
                best_depth = depth
                best_leaf = leaf
                best_split = split
print('Лучшая модель:')
print(f'max_depth={best_depth}, min_samples_split={best_split}, min_samples_leaf={best_leaf}')
model = best_tree_model
scoring(features_valid, target_valid)

Лучшая модель:
max_depth=5, min_samples_split=2, min_samples_leaf=4
F1 = 0.57 | AUC-ROC = 0.84


Уже лучше. Но до требуемых 0.59 не дотягивает.  
Вернёмся к случайному лесу и подберём для него условно оптимальные гиперпараметры. Гиперпараметры будем с подбирать случайным образом с помощью метода RandomizedSearchCV. Чтобы ячейка выполнялась за адекватное время, количество итераций ограничим небольшим числом - 20 итераций. 

In [30]:
%%time
model = RandomForestClassifier(random_state=12345)
params = {
    'n_estimators':[x for x in range(10, 100, 1)],
    'max_depth':[x for x in range(1, 20, 1)], 
    'min_samples_leaf':[x for x in range(1, 20, 1)]
}
grid = RandomizedSearchCV(model, params, scoring='f1', n_iter=20, random_state=12345)
grid.fit(features_train, target_train)
print('Лучший результат:')
print(grid.best_params_)
print('\nF1-мера на тренировочной выборке:')
print(grid.best_score_)
print('\nКачество модели на валидационной выборке:')
model = grid.best_estimator_
model.fit(features_train, target_train)
scoring(features_valid, target_valid)
print()

Лучший результат:
{'n_estimators': 82, 'min_samples_leaf': 4, 'max_depth': 13}

F1-мера на тренировочной выборке:
0.8811226145781107

Качество модели на валидационной выборке:
F1 = 0.60 | AUC-ROC = 0.86

CPU times: user 32.7 s, sys: 139 ms, total: 32.8 s
Wall time: 33.1 s


На тренировочной выборке модель раскачалась до F1 = 0.88. На валидации получили значение F1-меры 0.6, AUC-ROC 0.86. То есть модель переобучилась, что может быть связано с тем, что мы увеличили количество объектов положительного класса в 4 раза и модель одни и те же объекты встречала неоднократно. Однако на валидации получили результат F1 = 0.6, что соответствует поставленной задаче. Следующим шагом проверим модель на тестовой выборке. 

**Выводы:**  
- Исследовано три метода работы с несбалансированными класами: взвешивание, увеличение выборки и её уменьшение. 
- Взвешивание классов оказало значительное влияние на результат работы модели лоигстической регрессии, F1 увеличился с 0.36, до 0.5. На модели случайного леса в дефолтной постановке, взвешивание классов на модели не показало положительного эффекта.
- Наилучший результат для модели случайного леса на основании оценок F1 и AUC-ROC показало увеличение выборки положительного класса в 4 раза, то есть до достижения равенства классов, однако это вызвало переобучение модели. Уменьшение выборки отрицательного класса оказало некоторый положительный эффект при небольшом даунсэмплинге, но при дальнейшем уменьшении вызвало эффект недообучения. 
- После балансировки классов исследовано три алгоритма обучения и выбран наилучший - модель случайного леса. Для этой модели проведён случайный подбор гиперпараметров. Получен F1 = 0.6, AUC-ROC = 0.86

## Итоговое тестирование модели

Проверим работу модели на тестовой выборке, которая составляет 20% от исходного массива данных. 

In [31]:
scoring(features_test, target_test)

F1 = 0.63 | AUC-ROC = 0.87


**Вывод:**  
Результат удовлетворительный. 

## Заключение

- Выполнена работа по построению модели прогнозирования оттока клиентов "Бета-Банка" на основе исторических данных. 
- В процессе предварительной обработки выявлен дисбаланс классов целевого признака. Проведено исследование, которое показало, что наиболее эффективный способ борьбы с дисбалансом - увеличение выборки положительного класса до достижения паритета между классами. 
- Выбрана наилучшая модель - алгоритм "случайного леса". Для данной модели проведёт подбор гиперпараметров методом случайного поиска. 
- Оценка качества модели на валидационной выборки показало величину F1-меры 0.6, AUC-ROC 0.86. Итоговое тестирование показало результат F1 = 0.63, AUC-ROC = 0.87, что удовлетворяет требованию заказчика F1 > 0.59.